In [695]:
%autoreload 2
import pandas as pd
import os
from glob import glob
import numpy as np
from plotnine import *
import IPython
import librosa
import kaldi_io
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [605]:
train_data = pd.read_csv('../data/train_split_Depression_AVEC2017.csv')
dev_data = pd.read_csv('../data/dev_split_Depression_AVEC2017.csv')

In [701]:
transcripts = []
for fname in glob("labels_processed/Typed_TRANSCRIPT/*.csv"):
    cur_df = pd.read_csv(fname, sep = '\t')
    cur_df['Participant_ID'] = int(os.path.basename(fname).split('_')[0])
    transcripts.append(cur_df)
transcripts = pd.concat(transcripts).reset_index()
transcripts['value'].astype(str, copy=False)
transcripts['value'] = transcripts['value'].str.strip()
transcripts.dropna(inplace=True)

first_words = transcripts[(transcripts.speaker == 'Ellie') & (transcripts.type== 'interrogative')].value.str.split().apply(lambda x:x[0]).unique().tolist()
first_words += ['so','tell','who','has']

print("Estimated first words: {}".format(first_words))

# forgotten = ['so how are you doing today', "who's someone that's been a positive influence in your life"]

# All words starting with one of the first words in interrogaive
rightside = transcripts.value.apply(lambda x: any([x.startswith(a) for a in first_words]))
# Check that only ellie said that
leftside = transcripts.speaker == 'Ellie'
words1 = transcripts[leftside & rightside]
print(len(words1.value.unique()))
ellie_transcripts = transcripts[leftside]
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    IPython.display.display(ellie_transcripts[(~ellie_transcripts.isin(words1))].dropna().value.value_counts().to_frame())
# words1 = transcripts[(transcripts.speaker == 'Ellie') & (transcripts.value.str.split().apply(lambda x:x[0]).isin(first_words))].value


# transcripts[(transcripts.speaker == 'Ellie') & (~transcripts.value.isin(words1))].value.value_counts().to_frame()

Estimated first words: ['where', 'when', 'how', 'why', 'are', 'what', 'do', 'have', 'can', 'did', 'is', 'could', 'so', 'tell', 'who', 'has']
126


,value
okay,1057
yeah,684
mhm,545
that's good,473
uh huh,270
that's great,245
really,199
i see what you mean,194
goodbye,185
thanks for sharing your thoughts with me,181


In [589]:
for fname in glob('../data/*_TRANSCRIPT.csv'):
    cur_df = pd.read_csv(fname, sep = '\t')
    cur_df['Participant_ID'] = int(os.path.basename(fname).split('_')[0])
    transcripts.append(cur_df)
transcripts = pd.concat(transcripts).reset_index()
transcripts['value'].astype(str, copy=False)
transcripts.dropna(inplace=True)

transcripts[(transcripts.speaker == 'Ellie') & (transcripts.value.str.contains("_",regex=False))].head()
# transcripts.head(5)

,index,start_time,stop_time,speaker,value,Participant_ID
3,3,55.674,56.134,Ellie,okay_confirm (okay),477
4,4,57.244,59.514,Ellie,how_doingV (so how are you doing today),477
6,6,61.759,62.399,Ellie,thats_good (that's good),477
7,7,63.154,64.664,Ellie,great_thanks (i'm great thanks),477
8,8,65.423,66.743,Ellie,where_originally (where are you from originally),477


In [604]:
train_transcript = transcripts[transcripts['Participant_ID'].isin(train_data['Participant_ID'].unique())].reset_index(drop=True)
dev_transcript = transcripts[transcripts['Participant_ID'].isin(dev_data['Participant_ID'].unique())].reset_index(drop=True)
# train_transcript[(train_transcript.speaker == 'Ellie') & (train_transcript.value.str.contains("(",regex=False))].value.unique()
# train_transcript[(train_transcript.speaker == 'Ellie') ].value.str.match('^what|when')



# train_transcript = train_transcript[~((train_transcript.value.str.split().apply(len) <= 2) & (train_transcript.speaker == 'Ellie'))]
# for name, participant in train_transcript.groupby('Participant_ID'):
#     participant.groupby((participant.speaker != participant.speaker.shift(1)).cumsum()).value.apply('|'.join)
# train_transcript.groupby('Participant_ID').apply(lambda x: (x.speaker != x.speaker.shift(1)).cumsum())
# train_transcript.groupby((train_transcript['speaker'] != train_transcript['speaker'].shift(1)).cumsum()).value.apply('|'.join).to_frame().to_csv('dump.csv')
# speakers = train_transcript.speaker.reset_index()

array(["IntroV4Confirmation (hi i'm ellie thanks for coming in today i was created to talk to people in a safe and secure environment i'm not a therapist but i'm here to learn about people and would love to learn about you i'll ask a few questions to get us started and please feel free to tell me anything your answers are totally confidential are you ok with this)",
       'okay_confirm (okay)', 'how_doingV (so how are you doing today)',
       "thats_good (that's good)",
       'where_originally (where are you from originally)', 'cool3 (cool)',
       'like_about_LA (what are some things you really like about l_a)',
       'uh_huh (uh huh)',
       "dont_like_LA (what are some things you don't really like about l_a)",
       'yeah3 (yeah)',
       "travel_shoes (i'm sure you can tell by my shoes i'm not much of a world explorer do you travel a lot)",
       'mhm (mhm)', 'travel_changed (what do you enjoy about traveling)',
       'study (what did you study at school)',
       'still_w

In [518]:
for participant_id  in train_data['Participant_ID'].values:
    covarep_feature = np.genfromtxt('../data/{}_COVAREP.txt'.format(participant_id))
    print(covarep_feature.shape)
# data = {s:np.genfromtxt(s, delimiter=',') for s in glob('../data/*_COVAREP*')[:1]}

OSError: ../data/303_COVAREP.txt not found.

In [55]:
train_data.head()

,Participant_ID,PHQ8_Binary,PHQ8_Score,Gender,PHQ8_NoInterest,PHQ8_Depressed,PHQ8_Sleep,PHQ8_Tired,PHQ8_Appetite,PHQ8_Failure,PHQ8_Concentrating,PHQ8_Moving
0,303,0,0,0,0,0,0.0,0,0,0,0,0
1,304,0,6,0,0,1,1.0,2,2,0,0,0
2,305,0,7,1,0,1,1.0,2,2,1,0,0
3,310,0,4,1,1,1,0.0,0,0,1,1,0
4,312,0,2,1,0,0,1.0,1,0,0,0,0


In [33]:
sentences = [["cat", "say", "meow"], ["dog", "say", "woof"]]
model = Word2Vec(sentences, size=100, window=3, min_count=3, workers=4, seed=1)
model

RuntimeError: you must first build vocabulary before training the model